In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn 
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("../Data/single_dataset.csv", low_memory=False)
df.dropna(axis=0, how='any', inplace=True)

#enc characters
enc_p1 = LabelEncoder()
enc_p2 = LabelEncoder()

enc_p1.fit(df['CHAR_P1'])
enc_p2.fit(df['CHAR_P2'])
df['CHAR_P1'] = enc_p1.transform(df['CHAR_P1'])
df['CHAR_P2'] = enc_p2.transform(df['CHAR_P2'])

X_cols = ['CHAR_P1', 'CHAR_P2', 'S_airborne_P1',
   'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
   'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
   'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
   'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
   'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

y_cols = ['B_damage_P1',
   'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
   'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
   'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
   'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
   'B_triggers_logical_P1']

df['S_airborne_P1'] = df['S_airborne_P1'].astype(bool).astype(float)
df['S_airborne_P2'] = df['S_airborne_P2'].astype(bool).astype(float)

X = df[X_cols]
y = df[y_cols]

In [ ]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

In [ ]:
class FrameDataset(Dataset):
    def __init__(self,X,y):
        self.x_train=torch.tensor(X,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.y_train)
  
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out